In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install scattertext
!pip install spacy
!python -m spacy download en

     |████████████████████████████████| 7.3 MB 3.7 MB/s 
     |████████████████████████████████| 24.1 MB 2.7 kB/s 
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9310 sha256=497ee5fceb8060c5c784ea40abb086643036816516967a919e706d5c01716999
  Stored in directory: /root/.cache/pip/wheels/cb/19/58/4e8fdd0009a7f89dbce3c18fff2e0d0fa201d5cdfd16f113b7
Successfully built flashtext
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 12.0 MB 4.1 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
!pip install news-please
!pip install fuzzywuzzy
!pip install python-Levenshtein

     |████████████████████████████████| 67 kB 2.7 MB/s 
     |████████████████████████████████| 254 kB 9.1 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 3.4 MB 32.6 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 378 kB 54.4 MB/s 
     |████████████████████████████████| 211 kB 70.0 MB/s 
     |████████████████████████████████| 981 kB 72.7 MB/s 
     |████████████████████████████████| 47 kB 4.5 MB/s 
     |████████████████████████████████| 40 kB 5.1 MB/s 
     |████████████████████████████████| 3.7 MB 45.4 MB/s 
     |████████████████████████████████| 263 kB 44.9 MB/s 
     |████████████████████████████████| 79 kB 7.0 MB/s 
     |████████████████████████████████| 547 kB 67.2 MB/s 
     |████████████████████████████████| 8.0 MB 45.3 MB/s 
     |████████████████████████████████| 138 kB 53.7 MB/s 
     |████████████████████████████████| 87 kB 6.1 MB/s 
     |███████████████████████

In [ ]:
%matplotlib inline
import scattertext as st
import re, io
from pprint import pprint
import pandas as pd
import numpy as np
from scipy.stats import rankdata, hmean, norm
import spacy
import os, pkgutil, json, urllib
from urllib.request import urlopen
from IPython.display import IFrame
from IPython.core.display import display, HTML
from scattertext import CorpusFromPandas, produce_scattertext_explorer
display(HTML("<style>.container { width:98% !important; }</style>"))
nlp = spacy.load('en')

In [ ]:
def read_data(filename):
    # read in csv
    df = pd.read_csv(filename, encoding='utf-8')
    
    
    df = df.loc[:, ['Bias', 'Source', 'Text']]
    
    return df

In [ ]:
convention_df = read_data('/content/drive/MyDrive/articles_with_bias.csv')
convention_df.head()

,Bias,Source,Text
0,Right,Bridgemi,The Dearborn-based automaker chose Tennessee f...
1,Right,Bridgemi,"From a purely geographic perspective, most of ..."
2,Right,Bridgemi,“The focus of the last election is always goin...
3,Right,Bridgemi,"Marquette, Ottawa, Washtenaw and Kent county h..."
4,Right,Bridgemi,"The finalized legislation, brokered by Whitmer..."


In [ ]:
print("Document Count")
print(convention_df.groupby('Bias')['Text'].count())

Document Count
Bias
Center        463
Lean Left     570
Lean Right    118
Right         581
Name: Text, dtype: int64


In [ ]:
import re
import string

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# strip texts of punctuation, boilerplate, and stop words
def text_prepare(text):
    """
        text: a string
        return: modified initial string
    """
    text = text.lower()
    text = text.replace('\n',' ')
    
    letters = list(string.ascii_lowercase)
    numbers = ['0','1','2','3','4','5','6','7','8','9']
    banned = ["’","’","“","—","”","‘","–",'#','[','/','(',')','{','}','\\','[',']','|','@',',',';','+','-']
    banned = ''.join(banned) + string.punctuation + ''.join(numbers)
    banned = banned.replace(".", "")
    stop_list = set(stopwords.words('english') + letters)
    
    translation_table = dict.fromkeys(map(ord, banned), ' ')
    text = text.translate(translation_table)
    text = re.sub(' +',' ',text)
    text = ' '.join([word for word in text.split() if word not in stop_list])
    return text

In [ ]:
convention_df.dropna(subset=['Text'],inplace=True)

In [ ]:
convention_df = convention_df.sample(frac=1).reset_index(drop=True)

In [ ]:
# rewrite df with cleaned text
for i in range(0, len(convention_df)):
  convention_df.at[i,'text'] = text_prepare(convention_df.at[i,'Text'])
convention_df.head()

,Bias,Source,Text,text
0,Lean Left,The Star,As phase one of Ontario’s vaccine certificatio...,phase one ontario vaccine certification system...
1,Lean Right,Global News,I finally did it!\n\nAfter months of hard work...,finally months hard work included shadow swing...
2,Lean Left,The Star,"In a dramatic statement, Harvard University re...",dramatic statement harvard university recently...
3,Right,Channel News,Chieng jokingly described the watch’s two-tone...,chieng jokingly described watch two tone colou...
4,Center,BBC,"""For us to stay closed for this long without a...",us stay closed long without compensation disas...


In [ ]:

convention_df.groupby('Bias').apply(lambda x: x.Text.apply(lambda x: len(x.split())).sum())
convention_df['Parsed'] = convention_df.text.apply(nlp)

In [ ]:
corpus = st.CorpusFromParsedDocuments(convention_df[:200], category_col='Bias', parsed_col='Parsed').build()

In [ ]:
#list of words in corpus: corpus._term_idx_store
stop_word_list = ['via getty', 'inbox', 'subscribe', '×', 'close ×', 'screen close', 'full screen', 'buy second', 'second continue', 'story continued', 'llc permission', '―', 'xe', '\\xe2\\x80\\x99', 'news', 'for reprint', 'llc', 'post', 'click', 'to', '’ve', 'unsupported on', 'share', 'that ’s', 'still', 'got', 'it', '37', 'of his', 'this report', 'ofs', 'fox', 'photos', '’m', 'is the', 's.', 'around', 'times', 'also', 'the', 'copyright', 'washington times', 'mr', 'press', 'wait', 'associated', 'unsubscribe', 'view', 'photo wait', 'http', '#', 'associated press', 'more videos', 'get', 'just watched', 'permission', 'however', 'b.', 'ms.', 'here©', 'device', 'copyright ©', 'paste', '10', 'the associated', 'contributed to', 'hide', 'and his', 'videos', 'said mr.', '_', '©', 'contributed', 'embed', 'n’t', '/', 'something', 'i', 'that they', 'read', 'for a', 'playback', 'must watch', 'washington post', 'just', 'to get', 'r', 'read more', 'toggle', 'more', 'i ’m', 'follow', 'is', 'https', ' ', 'said', 'mr.', 'unsupported', 'or blog', 'your device', 'for', 'cnn', 'of 76', 'that', 'ms', 'andhis', 'click here', 'or share', 'replay', 'press contributed', 'they', 'must', 'prof', 'www', 'it ’s', 'told', '’re', 'the washington', '1', "'s rise", '© 2018', 'to this', 'skip', 'around the', 'blog', 'cut', 'told fox', 'mrs.', 'hide caption', 'ad', 'watched', '/ the', 'replay more', 'and the', '’s', '2018', 'copy', '&', 'read or', 'reprint permission', 'are', 'told cnn', 'watch', 'here for', 'also said', 'copy this', 'reprint', 'report', 'advertisement', 'mrs', 'caption', 'autoplay', 'fox news', 'dr', 'enlarge', 'times llc', '76', 'photo', 'this']
stop_word_list = list(set(stop_word_list))

In [ ]:
update_stop = []
for term in stop_word_list:
  if term in corpus._term_idx_store:
    update_stop.append(term)
corpus = corpus.remove_terms(update_stop)

In [ ]:
term_freq_df = corpus.get_term_freq_df()
term_freq_df.head()

,Lean Left freq,Lean Right freq,Right freq,Center freq
term,,,,
phase,3,0,2,2
one,115,23,123,69
ontario,84,2,0,0
vaccine,132,4,54,17
certification,4,0,0,0


In [ ]:
term_freq_df = corpus.get_term_freq_df()
list(term_freq_df.columns.values)

['Lean Left freq', 'Lean Right freq', 'Right freq', 'Center freq']

In [ ]:
print(list(corpus.get_scaled_f_scores_vs_background().index[:10]))

['covid', 'twitter', 'chalamet', 'unvaccinated', 'khamissa', 'coronavirus', 'youngkin', 'zatzman', 'alharthy', 'schiaparelli']


In [ ]:
term_freq_df = corpus.get_term_freq_df()
term_freq_df['Lean Left Score'] = \
corpus.get_scaled_f_scores('Lean Left')
pprint(list(term_freq_df.sort_values(by='Lean Left Score', ascending=False).index[0:25]))

['greens',
 'ml',
 'kenney',
 'zatzman',
 'certificates',
 'canadians',
 'cori',
 'qr',
 'sickkids',
 'yo',
 'mcminn',
 'provincial',
 'federal council',
 'atwin',
 'pastry',
 'rolling stone',
 'levin',
 'birklid',
 'click get',
 'qr code',
 'sophs',
 'vaccine certificate',
 'get fox',
 'annamie',
 'mordecai']


In [ ]:
term_freq_df['Right Score'] = \
corpus.get_scaled_f_scores('Right')
pprint(list(term_freq_df.sort_values(by='Right Score', 
                                      ascending=False).index[0:25]))


['wilkinson',
 'mike wilkinson',
 'sushi',
 'state reported',
 'confirmed suspected',
 'henna',
 'back positive',
 'suspected covid',
 'europe',
 'michigan reported',
 'schiaparelli',
 'state also',
 'coronavirus tests',
 'whitmer',
 'chalamet',
 'khamissa',
 'related michigan',
 'daily average',
 'additional covid',
 'week ago',
 'percent positive',
 'brady',
 'alharthy',
 'reported additional',
 'confirmed covid']


In [ ]:
term_freq_df['Lean Right Score'] = \
corpus.get_scaled_f_scores('Lean Right')
pprint(list(term_freq_df.sort_values(by='Lean Right Score', 
                                      ascending=False).index[0:25]))


['photo gallery',
 'calgary',
 'story continues',
 'continues advertisement',
 'tsilhqot',
 'open photo',
 'indigenous relations',
 'relations office',
 'goose',
 'stew',
 'white goose',
 'goose flying',
 'flying report',
 'antibody tests',
 'peigan',
 'antibody test',
 'trc',
 'north peigan',
 'exp realty',
 'realty',
 'exp',
 'poucette said',
 'poucette',
 'calls action',
 'cree']


In [ ]:
term_freq_df = corpus.get_term_freq_df()
term_freq_df['dem_precision'] = term_freq_df['Lean Left freq'] * 1./(term_freq_df['Lean Left freq'] + term_freq_df['Right freq']+term_freq_df['Lean Right freq'])
term_freq_df['dem_freq_pct'] = term_freq_df['Lean Left freq'] * 1./term_freq_df['Lean Left freq'].sum()
term_freq_df['dem_hmean'] = term_freq_df.apply(lambda x: (hmean([x['dem_precision'], x['dem_freq_pct']])
                                                                   if x['dem_precision'] > 0 and x['dem_freq_pct'] > 0 
                                                                   else 0), axis=1)                                                        
term_freq_df.sort_values(by='dem_hmean', ascending=False).iloc[:10]

,Lean Left freq,Lean Right freq,Right freq,Center freq,dem_precision,dem_freq_pct,dem_hmean
term,,,,,,,
.,160,17,97,63,0.583942,0.003358,0.006678
vaccine,132,4,54,17,0.694737,0.002770,0.005519
covid,129,9,337,21,0.271579,0.002707,0.005361
party,119,2,5,9,0.944444,0.002498,0.004982
one,115,23,123,69,0.440613,0.002414,0.004801
health,112,11,119,24,0.462810,0.002351,0.004677
people,111,21,109,101,0.460581,0.002330,0.004636
new,109,7,218,45,0.326347,0.002288,0.004543
paul,95,0,1,2,0.989583,0.001994,0.003980


In [ ]:
corpus

In [25]:
html = st.produce_scattertext_explorer(corpus,
                                       category='Lean Left',
                                       category_name='demоcratic',
                                       not_category_name='republican',
                                       minimum_term_frequency=5,
                                       width_in_pixels=1000,
                                       transform=st.Scalers.log_scale_standardize)
file_name = 'AllSides_Scattertext_Log11.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 700, height=500)

In [26]:
html = st.produce_scattertext_explorer(corpus,
                                       category='Right',
                                       category_name='democratic',
                                       not_category_name='republican',
                                       minimum_term_frequency=5,
                                       width_in_pixels=1000,
                                       transform=st.Scalers.log_scale_standardize)
file_name = 'AllSides_Scattertext_Log12.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 700, height=500)

In [27]:
html = st.produce_scattertext_explorer(corpus,
                                       category='Lean Right',
                                       category_name='democratic',
                                       not_category_name='republican',
                                       minimum_term_frequency=5,
                                       width_in_pixels=1000,
                                       transform=st.Scalers.log_scale_standardize)
file_name = 'AllSides_Scattertext_Log13.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 700, height=500)